# UNET Model for Classification

In [1]:
from importsfunctions import *

## Import train and validation dataset

In [2]:
DATAPATH = 'ISIC2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data_Resize_572_Augment/1/'
DATAPATH_segm = 'ISIC2017/ISIC-2017_Training_Part1_GroundTruth/ISIC-2017_Training_Part1_GroundTruth_Resize_572_Augment/1/'
DATAPATH_csv='ISIC2017/ISIC-2017_Training_Part3_GroundTruth.csv'
x_train,_,y_trainclass,nomeficht=import_datasetBSrot(DATAPATH,DATAPATH_segm, DATAPATH_csv,4000)
#x_train=x_train/255
DATAPATH = 'ISIC2017/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data_Resize_5721/'
DATAPATH_segm = 'ISIC2017/ISIC-2017_Validation_Part1_GroundTruth/ISIC-2017_Validation_Part1_GroundTruth_Resize_5721/'
DATAPATH_csv='ISIC2017/ISIC-2017_Validation_Part3_GroundTruth.csv'
x_validation,_,y_validationclass,nomefichv=import_datasetBSrot(DATAPATH,DATAPATH_segm, DATAPATH_csv,150)
x_validation=x_validation/255



rounded_train=np.argmax(y_trainclass, axis=1)
y_validationclass2=np.argmax(y_validationclass, axis=1)
class_weights = class_weight.compute_class_weight('balanced',np.unique(rounded_train),rounded_train)
class_weightsV = class_weight.compute_class_weight('balanced',np.unique(y_validationclass2),y_validationclass2)
classe_weight= {0 : 1.78253119, 1 : 2.62467192, 2 : 0.48590865 }
classe_weightV= {0 : 1.66666667, 1 : 1.19047619, 2 : 0.64102564 }
train_sample_weight = np.array([classe_weight[cls] for cls in rounded_train])
val_sample_weight = np.array([classe_weightV[cls] for cls in y_validationclass2])
data_gen_args = dict(rescale=1./255, horizontal_flip=True,vertical_flip=True )
datagen = ImageDataGenerator(**data_gen_args)
it = datagen.flow(x_train, y_trainclass, sample_weight=train_sample_weight, batch_size=8,shuffle=True)


yes
yes


## Create Model

### Teste 1:

In [ ]:
model_fixed = redeUNETclassfreeze(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
model_fixed.load_weights("BestSegmentatione.h5", by_name=True)
for layer in model_fixed.layers:
    layer.trainable= False
#model_fixed.summary()  

model_class = redeUNETclassfreeze2(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
#model_class.summary()  


model_final=Sequential()
model_final.add(model_fixed)
model_final.add(model_class)
model_final.compile(optimizer=Adam(learning_rate=1e-4, amsgrad=True ),
                    loss='categorical_crossentropy',metrics=['categorical_accuracy'],
                    weighted_metrics=['accuracy'])
model_final.summary()


Reduce=ReduceLROnPlateau(factor=0.7, patience=7, min_lr=1e-10, verbose=1)
filepath='Class_freezetotal11.h5'
filepath1='Class_freezetotal11ca.h5'
checkpointer = ModelCheckpoint(filepath,monitor='val_accuracy',save_weights_only=True, verbose=1, save_best_only=True)
checkpointer2 = ModelCheckpoint(filepath1,monitor='val_categorical_accuracy',save_weights_only=True, verbose=1, save_best_only=True)
earlystopper = EarlyStopping(patience=100, verbose=1)
log_dire = os.path.join(
    "logs",
    "freeze11","tudofreeze")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dire, histogram_freq=1,profile_batch=0)

results=model_final.fit_generator(it, steps_per_epoch=2000/8,
                    validation_data=(x_validation.astype(np.float32), y_validationclass.astype(np.float32),val_sample_weight),
                    epochs=200, callbacks=[checkpointer,checkpointer2,Reduce,tensorboard_callback],verbose=1)
plot_performanceClassification(results)

del model_final

model_fixed = redeUNETclassfreeze(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
model_class = redeUNETclassfreeze2(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)

model_final=Sequential()
model_final.add(model_fixed)
model_final.add(model_class)
model_final.compile(optimizer=Adam(learning_rate=1e-4, amsgrad=True ),
                    loss='categorical_crossentropy',metrics=['categorical_accuracy'],
                    weighted_metrics=['accuracy'])
#model_final.summary()
model_final.load_weights("Class_freezetotal11.h5")


predictions_val = model_final.predict(x_validation,batch_size=4)
classification_metrics(predictions_val,y_validationclass)

### Teste 3:

In [ ]:
model_fixed = redeUNETclassfreeze(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
model_class = redeUNETclassfreeze2(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
#model_class.summary()  

model_final=Sequential()
model_final.add(model_fixed)
model_final.add(model_class)
model_final.compile(optimizer=Adam(learning_rate=1e-5, amsgrad=True ),
                    loss='categorical_crossentropy',metrics=['categorical_accuracy'],
                    weighted_metrics=['accuracy'])
model_final.summary()
model_final.load_weights("Class_freezetotal10.h5")


Reduce=ReduceLROnPlateau(factor=0.7, patience=7, min_lr=1e-10, verbose=1)
filepath='Class_afterfreeze10.h5'
filepath1='Class_afterfreeze10ca.h5'
checkpointer = ModelCheckpoint(filepath,monitor='val_accuracy',save_weights_only=True, verbose=1, save_best_only=True)
checkpointer2 = ModelCheckpoint(filepath1,monitor='val_categorical_accuracy',save_weights_only=True, verbose=1, save_best_only=True)
earlystopper = EarlyStopping(patience=100, verbose=1)
log_dire = os.path.join(
    "logs",
    "freeze10","afterfreeze")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dire, histogram_freq=1,profile_batch=0)

results=model_final.fit_generator(it, steps_per_epoch=2000/8,
                    validation_data=(x_validation.astype(np.float32), y_validationclass.astype(np.float32),val_sample_weight),
                    epochs=200, callbacks=[checkpointer,checkpointer2,Reduce,tensorboard_callback],verbose=1)
plot_performanceClassification(results)

del model_final

model_fixed = redeUNETclassfreeze(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
model_class = redeUNETclassfreeze2(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
#model_class.summary()  

model_final=Sequential()
model_final.add(model_fixed)
model_final.add(model_class)
model_final.compile(optimizer=Adam(learning_rate=1e-4, amsgrad=True ),
                    loss='categorical_crossentropy',metrics=['categorical_accuracy'],
                    weighted_metrics=['accuracy'])
#model_final.summary()

model_final.load_weights("Class_afterfreeze.h5")


predictions_val = model_final.predict(x_validation,batch_size=4)
classification_metrics(predictions_val,y_validationclass)

### Teste 2

In [ ]:
model_fixed = redeUNETclassfreeze(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
model_fixed.load_weights("BestSegmentatione.h5", by_name=True)

model_class = redeUNETclassfreeze2(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
#model_class.summary()  


model_final=Sequential()
model_final.add(model_fixed)
model_final.add(model_class)
model_final.compile(optimizer=Adam(learning_rate=1e-4, amsgrad=True ),
                    loss='categorical_crossentropy',metrics=['categorical_accuracy'],
                    weighted_metrics=['accuracy'])
model_final.summary()


Reduce=ReduceLROnPlateau(factor=0.7, patience=7, min_lr=1e-8, verbose=1)
filepath='Class_unfreeze.h5'
filepath1='Class_unfreeze.h5'
checkpointer = ModelCheckpoint(filepath,monitor='val_accuracy',save_weights_only=True, verbose=1, save_best_only=True)
checkpointer2 = ModelCheckpoint(filepath1,monitor='val_categorical_accuracy',save_weights_only=True, verbose=1, save_best_only=True)
earlystopper = EarlyStopping(patience=100, verbose=1)
log_dire = os.path.join(
    "logs",
    "freeze3","nofreeze")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dire, histogram_freq=1,profile_batch=0)

results=model_final.fit_generator(it, steps_per_epoch=2000/8,
                    validation_data=(x_validation.astype(np.float32), y_validationclass.astype(np.float32),val_sample_weight),
                    epochs=200, callbacks=[checkpointer,checkpointer2,earlystopper,Reduce,tensorboard_callback],verbose=1)
plot_performanceClassification(results)

del model_final

model_fixed = redeUNETclassfreeze(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
model_class = redeUNETclassfreeze2(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)

model_final=Sequential()
model_final.add(model_fixed)
model_final.add(model_class)
model_final.compile(optimizer=Adam(learning_rate=1e-4, amsgrad=True ),
                    loss='categorical_crossentropy',metrics=['categorical_accuracy'],
                    weighted_metrics=['accuracy'])
#model_final.summary()
model_final.load_weights("Class_unfreeze.h5")



predictions_train = model_final.predict(x_train,batch_size=4)
classification_metrics(predictions_train,y_trainclass)

predictions_val = model_final.predict(x_validation,batch_size=4)
classification_metrics(predictions_val,y_validationclass)

### Teste 3 again

In [ ]:
model_fixed = redeUNETclassfreeze(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
model_class = redeUNETclassfreeze2(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
#model_class.summary()  

model_final=Sequential()
model_final.add(model_fixed)
model_final.add(model_class)
model_final.compile(optimizer=Adam(learning_rate=1e-5, amsgrad=True ),
                    loss='categorical_crossentropy',metrics=['categorical_accuracy'],
                    weighted_metrics=['accuracy'])
model_final.summary()
model_final.load_weights("Class_freezetotal11.h5")






Reduce=ReduceLROnPlateau(factor=0.7, patience=7, min_lr=1e-10, verbose=1)
filepath='Class_afterfreeze11.h5'
filepath1='Class_afterfreezeca11.h5'
checkpointer = ModelCheckpoint(filepath,monitor='val_accuracy',save_weights_only=True, verbose=1, save_best_only=True)
checkpointer2 = ModelCheckpoint(filepath1,monitor='val_categorical_accuracy',save_weights_only=True, verbose=1, save_best_only=True)
earlystopper = EarlyStopping(patience=150, verbose=1)
log_dire = os.path.join(
    "logs",
    "freeze11","afterfreeze")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dire, histogram_freq=1,profile_batch=0)

results=model_final.fit_generator(it, steps_per_epoch=2000/8,
                    validation_data=(x_validation.astype(np.float32), y_validationclass.astype(np.float32),val_sample_weight),
                    epochs=200, callbacks=[checkpointer,checkpointer2,Reduce,tensorboard_callback],verbose=1)
plot_performanceClassification(results)

del model_final

model_fixed = redeUNETclassfreeze(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
model_class = redeUNETclassfreeze2(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
#model_class.summary()  

model_final=Sequential()
model_final.add(model_fixed)
model_final.add(model_class)
model_final.compile(optimizer=Adam(learning_rate=1e-5, amsgrad=True ),
                    loss='categorical_crossentropy',metrics=['categorical_accuracy'],
                    weighted_metrics=['accuracy'])
#model_final.summary()

model_final.load_weights("Class_afterfreeze11.h5")


predictions_val = model_final.predict(x_validation,batch_size=4)
classification_metrics(predictions_val,y_validationclass)